<a href="https://colab.research.google.com/github/draculess99/Tensorflow2/blob/master/Section9_TensorFlow_Data_Validation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get install python-dev python-snappy

Reading package lists... Done
Building dependency tree       
Reading state information... Done
python-dev is already the newest version (2.7.15~rc1-1).
The following package was automatically installed and is no longer required:
  libnvidia-common-410
Use 'apt autoremove' to remove it.
The following NEW packages will be installed:
  python-snappy
0 upgraded, 1 newly installed, 0 to remove and 16 not upgraded.
Need to get 10.8 kB of archives.
After this operation, 39.9 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 python-snappy amd64 0.5-1.1build2 [10.8 kB]
Fetched 10.8 kB in 5s (2,025 B/s)
Selecting previously unselected package python-snappy.
(Reading database ... 130942 files and directories currently installed.)
Preparing to unpack .../python-snappy_0.5-1.1build2_amd64.deb ...
Unpacking python-snappy (0.5-1.1build2) ...
Setting up python-snappy (0.5-1.1build2) ...


In [1]:
!pip install -q tensorflow_data_validation

     |████████████████████████████████| 2.7MB 33.4MB/s 
     |████████████████████████████████| 2.7MB 44.6MB/s 
     |████████████████████████████████| 174kB 56.1MB/s 
     |████████████████████████████████| 51kB 16.8MB/s 
     |████████████████████████████████| 51kB 22.0MB/s 
     |████████████████████████████████| 1.0MB 46.1MB/s 
     |████████████████████████████████| 133kB 52.5MB/s 
     |████████████████████████████████| 81kB 24.6MB/s 
     |████████████████████████████████| 143kB 52.1MB/s 
     |████████████████████████████████| 92kB 28.0MB/s 
     |████████████████████████████████| 102kB 31.3MB/s 
     |████████████████████████████████| 163kB 49.7MB/s 
     |████████████████████████████████| 92kB 20.9MB/s 
ERROR: google-cloud-storage 1.16.1 has requirement google-cloud-core<2.0dev,>=1.0.0, but you'll have google-cloud-core 0.29.1 which is incompatible.
ERROR: google-cloud-translate 1.5.0 has requirement google-cloud-core<2.0dev,>=1.0.0, but you'll have google-cloud-core 0.29.1 w

In [0]:
import pandas as pd
import tensorflow as tf
import tensorflow_data_validation as tfdv

from __future__ import print_function

In [0]:
dataset = pd.read_csv("pollution-small.csv")

In [5]:
dataset.shape

(2188, 5)

In [0]:
training_data = dataset[:1600]

In [7]:
training_data.describe()

,pm10,no2,so2,soot
count,1600.000000,1600.000000,1600.000000,1600.000000
mean,49.656494,30.980519,16.229981,21.551956
std,35.211906,12.400788,10.621896,12.127354
min,6.380000,9.740000,4.010000,6.000000
25%,28.345000,22.567500,9.777500,14.400000
50%,38.835000,28.715000,13.275000,18.630000
75%,58.050000,36.370000,19.282500,24.072500
max,277.250000,138.010000,123.130000,107.650000


In [0]:
test_set = dataset[1600:]

In [9]:
test_set.describe()

,pm10,no2,so2,soot
count,588.000000,588.000000,588.000000,588.000000
mean,44.648248,37.296922,13.605170,18.441310
std,28.992087,10.940050,5.098944,6.596459
min,11.900000,15.070000,4.990000,8.000000
25%,28.337500,29.217500,10.122500,14.410000
50%,35.555000,35.815000,12.345000,17.090000
75%,50.812500,43.872500,15.855000,20.962500
max,273.770000,106.030000,38.030000,87.210000


In [0]:
### Generate training data statistics

In [10]:
train_stats = tfdv.generate_statistics_from_dataframe(dataframe=dataset)

W0707 15:41:02.075400 139825194850176 deprecation_wrapper.py:119] From /usr/local/lib/python2.7/dist-packages/tensorflow_transform/analyzers.py:948: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

W0707 15:41:03.574855 139825194850176 deprecation_wrapper.py:119] From /usr/local/lib/python2.7/dist-packages/tensorflow_transform/analyzers.py:994: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.



In [0]:
### Infering the schema

In [0]:
schema = tfdv.infer_schema(statistics=train_stats)

In [12]:
tfdv.display_schema(schema)

,Type,Presence,Valency,Domain
Feature name,,,,
'Date',BYTES,required,,-
'so2',FLOAT,required,,-
'pm10',FLOAT,required,,-
'no2',FLOAT,required,,-
'soot',FLOAT,required,,-


In [0]:
test_stats = tfdv.generate_statistics_from_dataframe(dataframe=test_set)

In [0]:
## Stage 4: Compare test statistics with the Schema

In [0]:
### Checking for anomalies in new data

In [0]:
anomalies = tfdv.validate_statistics(statistics=test_stats, schema=schema)

In [0]:
### Displaying all detected anomalies

- Integer larger than 10
- STRING type when expected INT type
- FLOAT type when expected INT type
- Integer smaller than 0

In [23]:
tfdv.display_anomalies(anomalies)

In [0]:
### New data WITH anomalies

In [0]:
test_set_copy = test_set.copy()

In [0]:
test_set_copy.drop("soot", axis=1, inplace=True)

In [0]:
### Statistics based on data with anomalies

In [0]:
test_set_copy_stats = tfdv.generate_statistics_from_dataframe(dataframe=test_set_copy)

In [0]:
anomalies_new = tfdv.validate_statistics(statistics=test_set_copy_stats, schema=schema)

In [29]:
tfdv.display_anomalies(anomalies_new)

,Anomaly short description,Anomaly long description
Feature name,,
'soot',Column dropped,Column is completely missing


In [0]:
## Stage 5: Prepare the schema for Serving

In [0]:
schema.default_environment.append("TRAINING")
schema.default_environment.append("SERVING")

In [0]:
### Removing a target column from the Serving schema

In [0]:
tfdv.get_feature(schema, "soot").not_in_environment.append("SERVING")

In [0]:
### Checking for anomalies between the SERVING environment and new test set

In [0]:
serving_env_anomalies = tfdv.validate_statistics(test_set_copy_stats, schema, environment="SERVING")

In [37]:
tfdv.display_anomalies(serving_env_anomalies)

In [0]:
# freezing the schema

In [0]:
tfdv.write_schema_text(schema=schema, output_path="pollution_schema.pbtxt")